# APAC Marketplace Monitoring Tool

In [318]:
#General config
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML
pd.set_option('display.max_colwidth', -1)

#AppNexus config
from link import lnk
mysql = lnk.dbs.mprod_api
vert = lnk.dbs.vertica

In [319]:
######################## APAC Seller Member List ################
apac_members = '(7418,7209,7114,2741,2742,2740,7151,2800,7179,6938,6899,7612,2539,3642,6860,7340,7262,7804,6866)'
#print (apac_members)
# 7080 FPT AdTrue, 2358 Ureka, 2007 Affinity.com has been removed
#1838 = Xaxis India


In [320]:
######################## List of APAC clients by Type (Direct vs. Indirect) ##########################
sql_andirect = """
SELECT m.id as member_id, billing_name, CASE WHEN (mg.created_on IS NOT NULL) THEN "AN Direct" ELSE "Indirect Supply" END as SMG,
date_format(mg.created_on,'%m/%d/%Y') as Add_Date
FROM api.member m LEFT JOIN common.member_seller_member_group mg on m.id=mg.member_id
WHERE m.id IN """ + apac_members + """
ORDER BY SMG, billing_name;
"""

df_andirect = mysql.select(sql_andirect).as_dataframe()
df_andirect



,member_id,billing_name,smg,add_date
0,6860,Baidu,AN Direct,01/13/2017
1,7612,Canvass Media,AN Direct,04/20/2017
2,3642,Cheetah Mobile,AN Direct,03/08/2016
3,7340,iHandy Network Limited,AN Direct,01/20/2017
4,6938,India Webportal Private Limited,AN Direct,09/21/2016
5,7151,Mobiventura Limited,AN Direct,01/31/2017
6,6899,NDTV Convergence Ltd,AN Direct,09/21/2016
7,7179,Postr Media Ltd.,AN Direct,11/28/2016
8,7804,Rajasthan Patrika Pvt Ltd,AN Direct,04/10/2017
9,2539,Times Internet Limited,AN Direct,03/08/2016


In [272]:
#Connects to Vertica - Get top 10 domains (by Geo) from each Seller last 30 days

top10_sql="""
SELECT * FROM 
(SELECT geo_country, m.billing_name, site_id, site_domain, application_id,
CASE WHEN audit_type IN (2,3) THEN 'Audited' WHEN audit_type IN (1,0) THEN 'Unaudited' END as audit_status,
sum(imps_seen) AS imps,
to_char(sum(imps_blacklist_or_fraud)/sum(imps_seen),'0.99') as 'blacklist_%',
RANK() OVER (PARTITION BY m.billing_name ORDER BY sum(imps_seen) DESC) AS imp_rank
FROM agg_platform_inventory_availability fact JOIN sup_api_member m ON fact.seller_member_id = m.id
WHERE 
  ymd >= now() - interval '31 days'
  AND seller_member_id IN """ + apac_members + """
  AND   geo_country IN ('JP','IN','CN','ID','PH','TW','TH','KR','MY','VN','HK','SG')
  -- AND site_domain <> '---'
GROUP BY 1,2,3,4,5,6
) ranked
WHERE imp_rank <= 10 -- top 10
ORDER BY imps desc;
"""

df_top10 = vert.select(top10_sql).as_dataframe()
#df_top10
#df_top10.head()



In [294]:
##### Top Unaudited Domains in APAC sellers - AN Direct vs. Indirect Supply 
# Note: greater than 1MM imps in 30 days

unaudited_sql="""
SELECT m.billing_name as member, m.id as member_id, site_domain, application_id,
CASE WHEN audit_type IN (1,0) THEN 'Unaudited' ELSE 'Error'END as audit_status,
sum(imps_seen) AS imps,
to_char(sum(imps_blacklist_or_fraud)/sum(imps_seen),'0.99') as 'blacklist_%'
FROM agg_platform_inventory_availability fact JOIN sup_api_member m ON fact.seller_member_id = m.id
WHERE 
  audit_type IN (1,0) -- unaudited
  AND ymd >= now() - interval '31 days'
  AND seller_member_id IN """ + apac_members + """
  AND   geo_country IN ('JP','IN','CN','ID','PH','TW','TH','KR','MY','VN','HK','SG')
  -- AND site_domain <> '---'
GROUP BY 1,2,3,4,5
HAVING sum(imps_seen) > 1000000
ORDER BY imps desc
LIMIT 100;
"""

df1 = vert.select(unaudited_sql).as_dataframe()
df2 = df_andirect

df_unaudit_out = pd.merge(df1, df2, how='left', on= 'member_id')
df_format = df_unaudit_out[['member','member_id','smg','site_domain', 'application_id','audit_status','imps','blacklist_%']]
df_format.sort_values(by=['smg','imps'],ascending=[True,False])
#df_format.to_csv('unaudited.csv',index=False,encoding='utf-8')



,member,member_id,smg,site_domain,application_id,audit_status,imps,blacklist_%
0,NDTV Convergence Ltd,6899,AN Direct,carandbike.com,---,Unaudited,8000200,0.03
3,NDTV Convergence Ltd,6899,AN Direct,gadgets360.com,---,Unaudited,1224600,0.02
1,"A1 Platform Co, Ltd",2800,Indirect Supply,ads-optima.com,---,Unaudited,4213400,0.00
2,Pokkt,7209,Indirect Supply,---,com.explorationbase.proversion1,Unaudited,2299400,0.00
4,Pokkt,7209,Indirect Supply,---,---,Unaudited,1035300,0.00


In [317]:
##### Top Blacklisted Site/Domains in APAC sellers - AN Direct vs. Indirect Supply 

blacklist_sql="""
SELECT m.billing_name as member, m.id as member_id, sum(imps) AS imps,
SUM(CASE WHEN blacklisted_imps = 1 THEN imps ELSE 0 END) as blacklist_imps,
to_char(sum(blacklisted_imps)/sum(imps),'0.99') as 'blacklist'
FROM agg_dw_advertiser_publisher_analytics_adjusted fact JOIN sup_api_member m ON fact.seller_member_id = m.id
WHERE 
  ymd >= now() - interval '7 days'
  AND imp_type NOT IN (7)
  AND seller_member_id IN """ + apac_members + """
GROUP BY 1,2
ORDER BY blacklist desc
"""

df_blacklist = vert.select(blacklist_sql).as_dataframe()
df_blacklist



,member,member_id,imps,blacklist_imps,blacklist
0,NDTV Convergence Ltd,6899,93320,0,9.78
1,Times Internet Limited,2539,7678583,0,1.94
2,DataWrkz Direct,2741,3274277,0,1.17
3,Cheetah Mobile,3642,160067895,0,0.08
4,Baidu,6860,226814,0,0.05
5,Mobiventura Limited,7151,3040110,0,0.05
6,Ants Online Advertising Solutions Pte Ltd,7114,49696032,0,0.03
7,Postr Media Ltd.,7179,1584263,0,0.02
8,Xaxis - India,1838,161135988,0,0.02
9,"A1 Platform Co, Ltd",2800,13478958,0,0.01


In [247]:
#####################################
######## MEMBER LEVEL CHECKS ########
#####################################

#member_id = '459'
domain = 'www.ndtv.com'


In [248]:
#SPO Investigation

#Expose Domains
sql_expose = """SELECT m.billing_name, seller_member_id, f.publisher_id, site_id, site_domain, expose_domains, rtb_eligible, sum(imps_seen)
FROM agg_platform_inventory_availability f
JOIN sup_api_member m ON f.seller_member_id = m.id
WHERE imp_type IN (6) AND ymd > now() - interval '7 days'
AND site_domain IN ('""" + domain + """')
GROUP BY 1,2,3,4,5,6,7;
"""

df_domain = vert.select(sql_expose).as_dataframe()
df_domain

,billing_name,seller_member_id,publisher_id,site_id,site_domain,expose_domains,rtb_eligible,sum
0,Tremor Video Inc.,3332,744512,1849983,www.ndtv.com,1,True,400
1,Tremor Video Inc.,3332,744585,1850659,www.ndtv.com,1,True,100
2,McCann Disciplines LTD.,1940,942747,2705165,www.ndtv.com,1,True,81100
3,Adgorithms,927,967170,2817808,www.ndtv.com,1,True,200
4,Taboola Inc,3364,823694,2283873,www.ndtv.com,1,True,1400
5,Teads France SAS,3334,751746,1877155,www.ndtv.com,1,True,11800
6,Defy Media,1613,696325,2452974,www.ndtv.com,1,True,100
7,Tremor Video Inc.,3332,744691,1851649,www.ndtv.com,1,False,1400
8,PubSquared LLC,1538,845161,2805550,www.ndtv.com,1,True,200
9,Tremor Video Inc.,3332,744934,1853233,www.ndtv.com,1,True,4600


In [264]:
#SPO-ed Domains by Seller
sql_spo = """SELECT
    CASE WHEN site_domain='---' THEN application_id ELSE site_domain END as site_app,
    CASE WHEN site_domain = '---' THEN 'application' ELSE 'site' END as type,
    seller_member_id, site_id, billing_name,
    TO_CHAR(SUM(imps_seen),'99,999,999,999') as imps_seen,
    TO_CHAR(SUM(CASE WHEN rtb_eligible='f' THEN imps_seen ELSE 0 END),'99,999,999,999') as 'RTB_eligible=f',
    TO_CHAR(SUM(CASE WHEN rtb_eligible='t' THEN imps_seen ELSE 0 END),'99,999,999,999') as 'RTB_eligible=t',
    TO_CHAR(SUM(CASE WHEN rtb_eligible='t' THEN imps_seen ELSE 0 END)/SUM(imps_seen)*100,'990%') as demand_access_rate
FROM
    agg_platform_inventory_availability LEFT JOIN sup_api_member ON id = seller_member_id
WHERE
    ymd = date(getdate()-31) AND (site_domain ='""" + domain + """')
GROUP BY 1,2,3,4,5 ORDER BY 6 desc"""

df_spo = vert.select(sql_spo).as_dataframe()
df_spo


,site_app,type,seller_member_id,site_id,billing_name,imps_seen,rtb_eligible=f,rtb_eligible=t,demand_access_rate
0,www.ndtv.com,site,459,892105,Rubicon,"3,585,700",0,"3,585,700",100%
1,www.ndtv.com,site,181,1906507,Google AdExchange,"2,318,100","2,318,100",0,0%
2,www.ndtv.com,site,310,2355426,Technorati Media,"1,132,900",0,"1,132,900",100%
3,www.ndtv.com,site,3053,1104338,FreeWheel SSP,"727,700","727,700",0,0%
4,www.ndtv.com,site,181,2642336,Google AdExchange,"414,200","414,200",0,0%
5,www.ndtv.com,site,1899,2525725,Zedo (ZINC),"280,900",0,"280,900",100%
6,www.ndtv.com,site,181,1909017,Google AdExchange,"199,600","199,600",0,0%
7,www.ndtv.com,site,1940,2391616,McCann Disciplines LTD.,"85,900",0,"85,900",100%
8,www.ndtv.com,site,1613,2852103,Defy Media,"85,300",0,"85,300",100%
9,www.ndtv.com,site,3332,1851649,Tremor Video Inc.,"72,400","72,400",0,0%


In [263]:
### MySQL SPO

mysql_spo = """
SELECT site_id, rtb_suspended, last_modified FROM common.site_marketplace_map WHERE site_id = 892105 GROUP BY 1,2;
"""
df_mysql_spo = mysql.select(mysql_spo).as_dataframe()
df_mysql_spo


,site_id,rtb_suspended,last_modified
0,892105,0,2017-03-24 21:41:23


In [230]:
















#####


# SCRATCH


#####


sql_vp ="""
SELECT VP.member_id as 'Seller Member ID', VPM.buyer_member_id as 'Buyer Member ID', M.name as 'Buyer Name', 
(CASE WHEN VPM.expose_publishers = 1 then 'Exposed' else 'Hidden' end) as 'Publisher', 
VPM.url_exposure as 'URLS', 
(CASE WHEN VPM.expose_universal_categories = 1 then 'Exposed' else 'Hidden' end) as 'Universal Categories', 
VPM.expose_custom_categories as 'Custom Categories', 
(CASE WHEN VPM.expose_tags = 1 then 'Exposed' else 'Hidden' end) as 'Placements' 
FROM visibility_profile VP LEFT JOIN common.visibility_profile_member VPM on VP.id = VPM.visibility_profile_id 
LEFT JOIN bidder.member M on VPM.buyer_member_id = M.id 
WHERE VP.member_id = 'field1' AND VPM.deleted = 0
"""
df = mysql.select(sql_vp).as_dataframe()
df


OperationalError: (2013, 'Lost connection to MySQL server during query')

In [ ]:
#### % of Blacklisted Impressions

In [ ]:
### SPO Activity
"""SELECT
    CASE WHEN site_domain='---' THEN application_id ELSE site_domain END as site_app,
    CASE WHEN site_domain = '---' THEN 'application' ELSE 'site' END as type,
    seller_member_id,
    billing_name,
    TO_CHAR(SUM(imps_seen),'99,999,999,999') as imps_seen,
    TO_CHAR(SUM(CASE WHEN rtb_eligible='f' THEN imps_seen ELSE 0 END),'99,999,999,999') as 'RTB_eligible=f',
    TO_CHAR(SUM(CASE WHEN rtb_eligible='t' THEN imps_seen ELSE 0 END),'99,999,999,999') as 'RTB_eligible=t',
    TO_CHAR(SUM(CASE WHEN rtb_eligible='t' THEN imps_seen ELSE 0 END)/SUM(imps_seen)*100,'990%') as demand_access_rate
FROM
    agg_platform_inventory_availability
    LEFT JOIN sup_api_member ON id = seller_member_id
WHERE
    ymd = date(getdate()-1)
    AND (site_domain = '$1' OR application_id = '$1')
GROUP BY    1,2,3,4
ORDER BY    5 desc LIMIT 20;"""

In [74]:
from datetime import datetime, timedelta
from sqlalchemy.engine import create_engine
import pandas as pd 
 
presto_hostname='02.bm-hadoopf-presto.prod.nym2'
presto_port='8080'
presto_catalog='hive'
presto_database='dmf'
 
engine = create_engine('presto://{presto_hostname}:{presto_port}/{presto_catalog}/{presto_database}'
                       .format(presto_hostname=presto_hostname,
                              presto_port=presto_port,
                              presto_catalog=presto_catalog,
                              presto_database=presto_database))
 
#Get the dh for yesterday at this hour and print the query
query_hour = (datetime.now()-timedelta(days=1)).strftime("%Y-%m-%d %H")
query = """
SELECT count(*)
FROM dmf.log_impbus_impressions_pq
where dh = '{query_hour}'
""".format(query_hour=query_hour)
print(query)
 
#Use pandas to access the table through sqlalchemy
df = pd.read_sql(query, engine)
print(df.to_string())

NoSuchModuleError: Can't load plugin: sqlalchemy.dialects:presto

In [ ]:
############## CODE SNIPPETS ###############

#df.to_csv('test.csv')



HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')